# Steps for Training
- Extract all the samples and generate spectrogram jpegs for each
- Train a CNN on the jpegs
- 

In [1]:
from utils import general_utils
import importlib
importlib.reload(general_utils)

train_files, train_targets, train_target_names = general_utils.load_dataset('data/nsynth-train-spectrograms')

print('info about training set')
print(len(train_files))
print(train_files[500:501])
print(train_targets[500:501])
print(train_target_names[500:501])

valid_files, valid_targets, valid_target_names = general_utils.load_dataset('data/nsynth-valid-spectrograms')

print('info about validation set')
print(len(valid_files))
print('categories are: ', set(valid_target_names))
print(valid_files[500:501])
print(valid_targets[500:501])
print(valid_target_names[500:501])

Using TensorFlow backend.


info about training set
283704
['data/nsynth-train-spectrograms/vocal/vocal_acoustic_029-059-025.jpg']
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]
['vocal']
info about validation set
12678
categories are:  {'keyboard', 'brass', 'reed', 'vocal', 'organ', 'mallet', 'bass', 'guitar', 'string', 'flute'}
['data/nsynth-valid-spectrograms/brass/brass_acoustic_006-076-025.jpg']
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
['brass']


In [2]:
#build dataframe that will be part of the image generator construction
#PART 1: zip the data
import pandas as pd

# merge the three arrays into an array of tupples
train_data = list(zip(train_files, train_targets, train_target_names))
valid_data = list(zip(valid_files, valid_targets, valid_target_names))

In [3]:
# PART 2: to limit the size of the datasets, uncomment / change the code here

train_data = train_data[:100000]
valid_data = valid_data[:4300]

In [4]:
# PART 3: Create the dataframes

train_df = pd.DataFrame(train_data, columns = ['file_paths', 'targets', 'target_names'])
valid_df = pd.DataFrame(valid_data, columns = ['file_paths', 'targets', 'target_names'])

In [5]:
print('train shape', train_df.shape)
print('valid shape', valid_df.shape)

train shape (100000, 3)
valid shape (4300, 3)


In [6]:
# create the data generator
from keras_preprocessing.image import ImageDataGenerator

datagen=ImageDataGenerator(rescale=1./255.,validation_split=0.25)

# make the training data generator
train_generator = datagen.flow_from_dataframe(
        dataframe=train_df,
        x_col='file_paths',
        y_col='target_names',
        batch_size=32,
        seed=69,
        shuffle=True,
        class_mode='categorical',
        target_size=(64,64)
    )


Found 100000 images belonging to 10 classes.


In [7]:
# make the validation data generator
valid_generator = datagen.flow_from_dataframe(
        dataframe=valid_df,
        x_col='file_paths',
        y_col='target_names',
        batch_size=32,
        seed=69,
        shuffle=True,
        class_mode='categorical',
        target_size=(64,64)
    )

Found 4300 images belonging to 10 classes.


In [8]:
# USEFULL THINGS

#to go through all of the files to make sure they're finable
#import os.path
#for f in valid_files:#valid_generator.filepaths:
#    if not os.path.exists(f):
#        print(f, 'does not exist')

# get all of the file paths in the data generator
#generator_files = set(valid_generator.filepaths)

# make sure all of the actual files are accounted for in the generator
#i = 0
#for f in valid_files:
#    if f not in generator_files:
#        print('N', f)
#        i+=1
    #else:
        #print('Y', f)
#print(i, 'files do not match')

In [9]:
# Inspired by https://medium.com/gradientcrescent/urban-sound-classification-using-convolutional-neural-networks-with-keras-theory-and-486e92785df4

from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=(64,64,3)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizers.rmsprop(lr=0.0005, decay=1e-6),loss="categorical_crossentropy",metrics=["accuracy"])
model.summary()

W0902 20:40:53.132492 139741414876992 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/magenta-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:68: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0902 20:40:53.150637 139741414876992 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/magenta-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:508: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0902 20:40:53.153576 139741414876992 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/magenta-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3837: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0902 20:40:53.181541 139741414876992 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/magenta-gpu/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3661: The name

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 62, 62, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 62, 62, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 64)        36928     
__________

In [10]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
#STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
print('train_generator.n//train_generator.batch_size is {}//{}'.format(train_generator.n, train_generator.batch_size))
print('valid_generator.n//valid_generator.batch_size is {}//{}'.format(valid_generator.n, valid_generator.batch_size))
print('STEP_SIZE_TRAIN, STEP_SIZE_VALID is {}, {}'.format(STEP_SIZE_TRAIN, STEP_SIZE_VALID))

train_generator.n//train_generator.batch_size is 100000//32
valid_generator.n//valid_generator.batch_size is 4300//32
STEP_SIZE_TRAIN, STEP_SIZE_VALID is 3125, 134


In [11]:
from keras.callbacks import ModelCheckpoint 
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.v1.hdf5', 
                               verbose=1, save_best_only=True)

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10,
                    callbacks=[checkpointer]
)
model.evaluate_generator(generator=valid_generator, steps=STEP_SIZE_VALID)

W0902 20:40:53.633869 139741414876992 deprecation.py:323] From /home/ec2-user/anaconda3/envs/magenta-gpu/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0902 20:40:53.925768 139741414876992 variables.py:2445] Variable *= will be deprecated. Use `var.assign(var * other)` if you want assignment to the variable value or `x = x * y` if you want a new python Tensor object.


Epoch 1/10
3125/3125 [==============================] - 619s 198ms/step - loss: 1.0778 - acc: 0.6105 - val_loss: 1.0974 - val_acc: 0.6663

Epoch 00001: val_loss improved from inf to 1.09736, saving model to saved_models/weights.best.v1.hdf5
Epoch 2/10
3125/3125 [==============================] - 543s 174ms/step - loss: 0.6885 - acc: 0.7587 - val_loss: 1.0511 - val_acc: 0.7083

Epoch 00002: val_loss improved from 1.09736 to 1.05106, saving model to saved_models/weights.best.v1.hdf5
Epoch 3/10
3125/3125 [==============================] - 542s 174ms/step - loss: 0.6051 - acc: 0.7920 - val_loss: 1.0209 - val_acc: 0.6792

Epoch 00003: val_loss improved from 1.05106 to 1.02092, saving model to saved_models/weights.best.v1.hdf5
Epoch 4/10
3125/3125 [==============================] - 542s 173ms/step - loss: 0.5889 - acc: 0.8022 - val_loss: 1.1566 - val_acc: 0.6799

Epoch 00004: val_loss did not improve from 1.02092
Epoch 5/10
3125/3125 [==============================] - 542s 174ms/step - loss:

[1.00830357752901, 0.6572164948453608]